# **Step 1: Notebook to Create ML Model**
Predict **penguin species** given measurements of the bill, flippers, and body mass, and knowledge about penguin's sex and location

### **Installing `scikit-learn`**

1. To install `scikit-learn` using `pip`, run the following command: `pip install -U scikit-learn`
2. To install `scikit-learn` using `conda`, run the following command: `conda install -c conda-forge scikit-learn`

In [65]:
!pip install -U scikit-learn
#conda install -c conda-forge scikit-learn

---


### **Import Libraries**

In [66]:
import pandas as pd                  # Pandas
import numpy as np                   # Numpy
from matplotlib import pyplot as plt # Matplotlib
import seaborn as sns                # Seaborn

# Package to implement Decision Tree Model
import sklearn
from sklearn.tree import DecisionTreeClassifier

# Package for data partitioning
from sklearn.model_selection import train_test_split

# Package to visualize Decision Tree
from sklearn import tree

# Package for generating confusion matrix
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Package for generating classification report
from sklearn.metrics import classification_report

# Module to save and load Python objects to and from files
import pickle 

%matplotlib inline

# Display inline plots as vector-based (svg)
%config InlineBackend.figure_formats = ['svg']

### **Load Dataset as DataFrame**

In [67]:
airline_df = pd.read_csv('airline.csv')
airline_df.head()

,satisfaction,customer_type,age,type_of_travel,class,flight_distance,seat_comfort,departure_arrival_time_convenient,food_and_drink,gate_location,...,online_support,ease_of_online_booking,on-board_service,leg_room_service,baggage_handling,checkin_service,cleanliness,online_boarding,departure_delay_in_minutes,arrival_delay_in_minutes
0,dissatisfied,Loyal Customer,48,Personal Travel,Eco,4001,1,1,1,1,...,1,1,4,1,1,4,2,1,16,0
1,dissatisfied,Loyal Customer,48,Personal Travel,Eco,3980,1,1,1,1,...,4,4,2,3,2,3,2,4,0,0
2,dissatisfied,Loyal Customer,40,Personal Travel,Eco,2251,1,1,1,1,...,1,1,3,3,2,1,1,1,0,8
3,dissatisfied,Loyal Customer,46,Personal Travel,Eco,2453,1,1,1,1,...,5,3,1,4,1,2,2,3,3,15
4,dissatisfied,Loyal Customer,63,Personal Travel,Eco,2011,1,1,1,1,...,5,5,1,5,2,1,2,5,25,25


In [68]:
# Dropping null values
airline_df.dropna(inplace = True)
airline_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119255 entries, 0 to 119254
Data columns (total 22 columns):
 #   Column                             Non-Null Count   Dtype 
---  ------                             --------------   ----- 
 0   satisfaction                       119255 non-null  object
 1   customer_type                      119255 non-null  object
 2   age                                119255 non-null  int64 
 3   type_of_travel                     119255 non-null  object
 4   class                              119255 non-null  object
 5   flight_distance                    119255 non-null  int64 
 6   seat_comfort                       119255 non-null  int64 
 7   departure_arrival_time_convenient  119255 non-null  int64 
 8   food_and_drink                     119255 non-null  int64 
 9   gate_location                      119255 non-null  int64 
 10  inflight_wifi_service              119255 non-null  int64 
 11  inflight_entertainment             119255 non-null  

In [69]:
# Distribution of satisfaction column
airline_df['satisfaction'].value_counts(normalize = True)

satisfaction
satisfied       0.541436
dissatisfied    0.458564
Name: proportion, dtype: float64

### **Select Input and Output Features**

In [70]:
# Output column for prediction
output = airline_df['satisfaction'] 

# Input features (excluding satisfaction column)
features = airline_df[['customer_type', 'type_of_travel', 'class', 
                       'age', 'flight_distance', 'departure_delay_in_minutes',
                       'arrival_delay_in_minutes','seat_comfort','food_and_drink', 'gate_location'
                       ,'inflight_wifi_service',
                       'inflight_entertainment','online_support',
                       'ease_of_online_booking','on-board_service',
                       'leg_room_service','baggage_handling',
                       'checkin_service','cleanliness','online_boarding',
                       'departure_arrival_time_convenient'           
 ]]



### **One-hot Encoding to Handle Categorical Variables**

`pd.get_dummies()` is a Pandas function that converts categorical variables into numeric (dummy) variables.

In [71]:
# cat_var = [
#     'customer_type', 'type_of_travel', 'class',
#     'seat_comfort', 'food_and_drink','gate_location' , 'inflight_wifi_service',
#     'inflight_entertainment', 'online_support', 'ease_of_online_booking',
#     'on-board_service', 'leg_room_service', 'baggage_handling',
#     'checkin_service', 'cleanliness', 'online_boarding',
#     'departure_arrival_time_convenient'
# ]
cat_var = ['customer_type','type_of_travel','class']
features_encoded = pd.get_dummies(features, columns = cat_var)

features_encoded.head()

,age,flight_distance,departure_delay_in_minutes,arrival_delay_in_minutes,seat_comfort,food_and_drink,gate_location,inflight_wifi_service,inflight_entertainment,online_support,...,cleanliness,online_boarding,departure_arrival_time_convenient,customer_type_Loyal Customer,customer_type_disloyal Customer,type_of_travel_Business travel,type_of_travel_Personal Travel,class_Business,class_Eco,class_Eco Plus
0,48,4001,16,0,1,1,1,1,1,1,...,2,1,1,True,False,False,True,False,True,False
1,48,3980,0,0,1,1,1,4,1,4,...,2,4,1,True,False,False,True,False,True,False
2,40,2251,0,8,1,1,1,1,1,1,...,1,1,1,True,False,False,True,False,True,False
3,46,2453,3,15,1,1,1,3,1,5,...,2,3,1,True,False,False,True,False,True,False
4,63,2011,25,25,1,1,1,5,1,5,...,2,5,1,True,False,False,True,False,True,False


### **Data Partitioning**

In [72]:
train_X, test_X, train_y, test_y = train_test_split(features_encoded, output, test_size = 0.2, random_state = 1) 

- `test_size`: Amount of data to be used for testing.

- `random_state`: Sets a seed for random number generation, ensuring the split is reproducible (same train/test sets every time you run it)

## **Prediction Modeling using Decision Tree**

In [73]:
# Defining prediction model
clf = DecisionTreeClassifier(random_state = 0)

# Fitting model on training data
clf.fit(train_X, train_y)

,criterion,'gini'
,splitter,'best'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,None
,random_state,0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,class_weight,None


### **Visualizing Trained Decision Tree Model**

In [74]:
# fig = plt.figure(figsize = (25,20))
# a = tree.plot_tree(decision_tree = clf,
#                    max_depth = None,
#                    feature_names = train_X.columns,
#                    class_names = clf.classes_,
#                    filled = True)

# # Saving Decision Tree
# fig.savefig("dt_visual.svg", bbox_inches = 'tight')

### **Model Evaluation**

#### **Evaluation on Train Set**

In [75]:
# Predictions on training set
y_pred_train = clf.predict(train_X)

# Now generate confusion matrix
cm = confusion_matrix(train_y, y_pred_train, labels = clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = clf.classes_)

# Specify figure size
fig, ax = plt.subplots(figsize = (5, 5))
plt.rcParams.update({'font.size': 12})

# Display Confusion Matrix
disp.plot(cmap = 'PuBu', ax = ax);

#### **Evaluation on Test Set**

In [76]:
# Predictions on test set
y_pred = clf.predict(test_X)

# Now generate confusion matrix
cm = confusion_matrix(test_y, y_pred, labels = clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix = cm, display_labels = clf.classes_)

# Specify figure size
fig, ax = plt.subplots(figsize = (5, 5))
plt.rcParams.update({'font.size': 12})

# Display Confusion Matrix
disp.plot(cmap = 'PuRd', ax = ax)

# Save as SVG
plt.savefig("confusion_mat.svg", bbox_inches = 'tight');

**Choosing Colormaps in Matplotlib**: [**Check this link**](https://matplotlib.org/stable/tutorials/colors/colormaps.html)

#### **Classification Report**

In [77]:
report = classification_report(test_y, y_pred, output_dict = True)
report_df = pd.DataFrame(report)
report_df

# Save the report as a CSV File
report_df.to_csv('class_report.csv') 

#### **Feature Importance Analysis**

It helps answer questions like:
**“Which features are most important in determining the output?”**

In [78]:
# Storing importance values from the trained model
importance = clf.feature_importances_

# Storing feature importance as a dataframe
feature_imp = pd.DataFrame(list(zip(train_X.columns, importance)),
               columns = ['Feature', 'Importance'])

feature_imp = feature_imp.sort_values('Importance', ascending = False).reset_index(drop = True)

# Bar plot
plt.figure(figsize = (10, 5))
plt.barh(feature_imp['Feature'], feature_imp['Importance'], color = ['purple', 'pink'])

plt.xlabel("Importance")
plt.ylabel("Input Feature")
plt.title('Which features are the most important for satisfaction prediction?') 
plt.tight_layout()
plt.savefig("feature_imp.svg");

## **Save Your Prediction Model**

**Using Pickle**

 <img src="https://d33wubrfki0l68.cloudfront.net/0f66116c250285870171e38f309161bafc25f06b/dbd4c/assets/images/posts/pickled-cabbage.jpg" width="350" style="float: center"/>

**Pickle** is a useful Python tool that allows you to save your ML models, to minimise lengthy re-training and allow you to share, commit, and re-load pre-trained machine learning models.

In [79]:
# Pickle file: saving the trained DT model
# Creating the file where we want to write the model
dt_pickle = open('dt_airline.pickle', 'wb') 

# Write DT model to the file
pickle.dump(clf, dt_pickle) 

# Close the file
dt_pickle.close() 